# Travel Advisor Code
### This code uses the TravelAdvisor API through RapidAPI to create a dataframe of the most relevant reviews, address and name of resturants from a set of locations in Canada. It then outputs the dataframe into a .csv to be used in other notebooks.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from pprint import pprint
from config import khem_rapid_key
import requests

In [8]:

#Travel Advisor
lat_url = "https://travel-advisor.p.rapidapi.com/restaurants/list-by-latlng"
locations = [[43.651070,-79.347015],[45.5019,-73.5674],[51.0447,-114.0719]]
#              ,[45.4215,-75.6972],[53.5461,-113.4937],[49.8954,-97.1385],
#              [43.5890,-79.6441],[49.2827,-123.1207],[43.7315,-79.7624],
#             [43.2557,-79.8711]]

location_id = []
res_name = []
res_add = []
review_dates = []


for locs in locations:
    lat_querystring = {"latitude":locs[0],"longitude":locs[1],"limit":"30","currency":"CAD","distance":"2","lunit":"km","lang":"en_US"}

    headers = {
    "X-RapidAPI-Key": khem_rapid_key,
    "X-RapidAPI-Host": "travel-advisor.p.rapidapi.com"
    }

    response = requests.get(lat_url, headers=headers, params=lat_querystring)



    for i in range(len(response.json()["data"])):
        location_id.append(response.json()["data"][i]["location_id"])

    loc_url = "https://travel-advisor.p.rapidapi.com/restaurants/get-details"

    for loc_id in location_id:
        loc_querystring = {"location_id":loc_id,"currency":"CAD","lang":"en_US"}

        resturants = requests.get(loc_url, headers=headers, params=loc_querystring)

        if "address" in resturants.json():
            res_name.append(resturants.json()["name"])
            res_add.append(resturants.json()["address"])

            if int(resturants.json()["num_reviews"]) > 0:
                review_dates.append(resturants.json()["reviews"][0]["published_date"])
            else:
                review_dates.append("")



data = {
    "Name" : res_name,
    "Address" : res_add,
    "Reviews" : review_dates
    
    
}


df = pd.DataFrame(data)

df_filtered = df[df['Reviews'] != ""]

df_filtered
df_filtered.to_csv("output/Travel_Advisor.csv")